In [ ]:
from datasets import load_from_disk
import torch
from transformers import T5Tokenizer
!pip install accelerate -U
import accelerate
print(accelerate._version_)
pip install accelerate transformers torch datasets
# Load the tokenized datasets
tokenized_train_dataset = load_from_disk('tokenized_datasets/train')
tokenized_val_dataset = load_from_disk('tokenized_datasets/val')
tokenized_test_dataset = load_from_disk('tokenized_datasets/test')

In [ ]:
# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small', legacy=False)

# Function to adjust padding
def adjust_padding(examples, max_length=512):
    # Adjust inputs
    inputs = tokenizer.pad(
        {"input_ids": examples["input_ids"], "attention_mask": examples["attention_mask"]},
        padding="max_length",
        max_length=max_length,
        return_tensors="pt"
    )
    # Adjust labels
    labels = tokenizer.pad(
        {"input_ids": examples["labels"]},
        padding="max_length",
        max_length=max_length,
        return_tensors="pt"
    )
    # Convert tensors to lists
    examples["input_ids"] = inputs["input_ids"].tolist()
    examples["attention_mask"] = inputs["attention_mask"].tolist()
    examples["labels"] = labels["input_ids"].tolist()
    return examples

# Adjust padding for each dataset
max_length_article = 512
max_length_summary = 150

tokenized_train_dataset = tokenized_train_dataset.map(lambda examples: adjust_padding(examples, max_length=max_length_article), batched=True)
tokenized_val_dataset = tokenized_val_dataset.map(lambda examples: adjust_padding(examples, max_length=max_length_article), batched=True)
tokenized_test_dataset = tokenized_test_dataset.map(lambda examples: adjust_padding(examples, max_length=max_length_article), batched=True)